# `CLOUD.jl` - 2D linear advection example

In [22]:
using CLOUD, OrdinaryDiffEq, TimerOutputs

Define the physical problem

In [23]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

Set discretization parameters

In [24]:
M = 4
p = 4
p_map = 4
form = WeakConservationForm(mapping_form=SkewSymmetricMapping(), inviscid_numerical_flux=LaxFriedrichsNumericalFlux(0.0))
strategy = PhysicalOperator()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.01*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [25]:
#=
reference_approximation = ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=50,
    operator_algorithm=DefaultOperatorAlgorithm())

reference_approximation = ReferenceApproximation(
    DGMulti(p), Tri(), operator_algorithm=BLASAlgorithm())
=#
reference_approximation = ReferenceApproximation(
    CollapsedModal(p), Tri())


mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.2)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), strategy,
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [26]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy);

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

CLOUD_reset_timer!()
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
            Thread 1                    Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            20.8s /  91.9%           16.8GiB /  96.6%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 local residual          815k    15.4s   80.5%  18.9μs   12.4GiB   76.3%  15.9KiB
   apply operators       815k    4.86s   25.4%  5.97μs   2.51GiB   15.5%  3.23KiB
     volume terms        815k    2.21s   11.6%  2.71μs   1.29GiB    8.0%  1.66KiB
     facet terms         815k    972ms    5.1%  1.19μs    572MiB    3.5%     736B
   eval nodal solution   815k    2.90s   15.2%  3.56μs   2.71GiB   16.7%  3.48KiB
   gather ext state      815k    2.24s   11.7%  2.75μs   3.52GiB   21.8%  4.53KiB
   eval num flu

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [27]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.027463443251205612
Conservation (initial/final/diff):
[4.14616793033111e-7][4.1461679213625896e-7][-8.968520370800093e-16]
Energy (initial/final/diff):
[0.1249918923475991][0.12499189234758748][-1.16157083951407e-14]


Write output to VTK file

In [28]:
postprocess_vtk(spatial_discretization, string(results_path,"solution.vtu"), last(sol.u));